# Рекомендательная система фильмов

---
**Цель**: Построить рекомендательную систему, которая будет предлагать пользователю список из *n* самых интересных для него фильмов. 
___
**Задачи**: 

1. Обработать данные.
2. Реализовать систему извлечения пула фильмов из которых будут выбираться *n* самых интересных.
3. Построить алгоритм матричного разложения для предсказания значения оценки.
4. Протестировать модель.
---

## Про датасет

Датасет взят с kaggle (https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset#). Он был собран GroupLens Research Project с сайта **MovieLens**. Он содержит данные о 100,000 рейтингах от 943 пользователей на 1682 фильмах. Также представлена простая демографическая информация о пользователях (возраст, пол, должность, почтовый индекс).

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872

## Источники

1. Kaggle (https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset#).
2. Учебник по машинному обучению от ШАДа (https://education.yandex.ru/handbook/ml).

## Метод решения.

&emsp;Для предсказаний рейтинга я собираюсь использовать метод Matrix factorization machines. 

## EDA

In [1]:
import pandas as pd

In [8]:
PATH_TO_DATA = 'ml-100k/'

In [12]:
ratings = pd.read_csv(
    filepath_or_buffer=PATH_TO_DATA + 'u.data',
    sep='\t',
    header=None,
    )
ratings.columns = ["user_id", "movie_id", "rating", "timestamp"]

print(ratings.info())
ratings.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   movie_id   100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
None


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
movies = pd.read_csv(
    filepath_or_buffer=PATH_TO_DATA + 'u.item',
    sep='\t',
    header=None
    )
movies =